In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
import shutil
import datetime
import matplotlib.pyplot as plt
import tabulate
from IPython.display import HTML, display, Markdown
import csv
import unicodedata
import getopt
from ipywidgets import HBox, VBox, Layout
from matplotlib.ticker import MaxNLocator

import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
import graphviz
from GraphVizUtil import *
from GraphVizFormats import *
import warnings
warnings.filterwarnings('ignore')
import seaborn as sb
sb.set(style='darkgrid', font_scale=1.2)

def filter_descriptions(tech_comm_desc):
    try:
        tech_comm_desc = tech_comm_desc.values[0][0].replace('#', '').replace('"','').replace("\n",'').strip()
    except:
        tech_comm_desc = 'No description provided'
    return tech_comm_desc

def create_args_flowd(df_graph):
    nodes, tech, ltech, to_tech, from_tech = set(), set(), set(), set(), set()
    for ind,row in df_graph.iterrows():
        #descriptions:
        input_comm_des = filter_descriptions(pd.read_sql("SELECT comm_desc FROM commodities WHERE comm_name='" + row['input_comm'] + "'", con))
        output_comm_des = filter_descriptions(pd.read_sql("SELECT comm_desc FROM commodities WHERE comm_name='" + row['output_comm'] + "'", con))
        tech_des = filter_descriptions(pd.read_sql("SELECT tech_desc FROM technologies WHERE tech='" + row['tech'] + "'", con))

        if 'ethos' in row['input_comm']:
            ltech.add('"' + row['tech'] + '"' +  ' [tooltip = "' + tech_des + '"]')
        else :
            nodes.add('"' + row['input_comm'] + '"' +  ' [tooltip = "' + input_comm_des + '"]')
        nodes.add('"' + row['output_comm'] + '"' +  ' [tooltip = "' + output_comm_des + '"]')
        tech.add('"' + row['tech'] + '"' +  ' [tooltip = "' + tech_des + '"]')

        if row['input_comm'] != 'ethos':
            to_tech.add('"%s"' % row['input_comm'] + '\t->\t"%s"' % row['tech']) 
        from_tech.add('"%s"' % row['tech'] + '\t->\t"%s"' % row['output_comm'])
    args = dict(
    enodes = "".join('%s;\n\t\t' % x for x in nodes),
    tnodes = "".join('%s;\n\t\t' % x for x in tech),
    iedges = "".join('%s;\n\t\t' % x for x in to_tech),
    oedges = "".join('%s;\n\t\t' % x for x in from_tech),
    snodes = ";".join('%s' %x for x in ltech),
    )
    return args


def return_format_colors():
    colors = {}
    colors.update(getColorConfig(False))
    return colors, quick_run_dot_fmt

def return_flowd_table(final_dem, level=1):
    df = pd.read_sql("SELECT * FROM Efficiency", con)
    df_sel = df[df['output_comm']==final_dem]
    if len(df_sel)==0:
        df_sel = df[df['tech']==final_dem]
    inputs = df_sel['input_comm'].unique()
    iterval=0
    if level!=0:
        while len(inputs)>0:
            df_append = df[df['output_comm'].isin(inputs)]
            df_sel = pd.concat([df_sel, df_append])
            inputs = df_append['input_comm'].unique()
            iterval+=1
            if iterval>level-1:
                break
    df_graph = df_sel[['input_comm', 'tech', 'output_comm']].drop_duplicates()
    return df_graph

def return_flowd_table_fwds(final_dem):
    df = pd.read_sql("SELECT * FROM Efficiency", con)
    df_sel = df[df['output_comm']==final_dem]
    if len(df_sel)==0:
        df_sel = df[df['tech']==final_dem]
    inputs = df_sel['input_comm'].unique()
    outputs = df_sel['output_comm'].unique()

    iterval=0
    while len(inputs)>0:
        df_append = df[df['output_comm'].isin(inputs)]
        df_sel = pd.concat([df_sel, df_append])
        inputs = df_append['input_comm'].unique()
        iterval+=1
        if iterval>2:
            break
    iterval=0
    while len(outputs)>0:
        df_append = df[df['input_comm'].isin(outputs)]
        df_sel = pd.concat([df_sel, df_append])
        outputs = df_append['output_comm'].unique()
        iterval+=1
        if iterval>=0:
            break

    df_graph = df_sel[['input_comm', 'tech', 'output_comm']].drop_duplicates()
    return df_graph

con = sqlite3.connect(r'../US_Regional.sqlite') #change path to database
cur = con.cursor()   
con.text_factory = str 

def controls_rows(w):
    controls = HBox(w.children[:-1], layout = Layout(flex_flow='row wrap', width='max-content'))
    output = w.children[-1]
    display(VBox([controls, output],  layout = Layout(flex_flow='columns wrap', width='max-content', size=10)))
    

### Table of Contents

[1. Buildings Sector Overview](#buildings)
<br>
&ensp;[1.1 Service Demands](#service_demands)
<br>
&emsp;[1.1.1 Annual Demand Projections](#annual_demands)
<br>
&emsp;[1.1.2 Hourly Demand Representation](#hourly_demands)
<br>
&ensp;[1.2 Technology/commodity description lookup tool](#description_look_up)
<br>
&ensp;[1.3 Network diagram lookup tool](#network_look_up)
<br>
&ensp;[1.4 Technology/commodity look-up tool](#lookup)
<br>
&ensp;[1.5 Demand Technology Specficiation](#demand_technology)
<br>
&emsp;[1.5.1 Investment Costs](#costinvest)
<br>
&emsp;[1.5.2 Efficiency](#efficiency)
<br>
&emsp;[1.5.3 Existing Capacity](#exist_cap)
<br>
&emsp;[1.5.4 Discount Rate](#disc_rates)




### 1. Buildings Sector Overview <a class="anchor" id="buildings"></a>
The buildings sector in the OEO database is broken down into the residential and commercial sectors. Both of these sectors consist of two key components: end-use service demands and the demand technologies used to meet those demands. The service demands in turn consist of two components: the annual regional demands projected to 2050, and the disaggregation of those annual demands to the hourly level so that they match the representative days modeled within the OEO database. The demand technologies are characterized by their capital costs, fixed and variable operation and maintenance costs, efficiencies, capacity factors, and fuel inputs.

#### 1.1 Service Demands <a class="anchor" id="service_demands"></a>
The <a href="https://www.nrel.gov/analysis/electrification-futures.html" rel="nofollow">NREL Electrification Futures Study</a> has used the EnergyPATHWAYS tool to develop a set of energy demand and supply projections for the buildings sector (and other sectors) to 2050. These annual projections are available at the state level for subsectors such as space heating, air conditioning, cooking, and water heating. The study also has projections of hourly electricity demands for some of these subsectors, by state. We are currently using these two datasets to develop demand projections for the OEO database. In the future, we plan to utilize more detailed projections based on the <a href="https://im3.pnnl.gov/model?model=BEND" rel="nofollow">Building ENergy Demand (BEND) Model</a>.


##### 1.1.1 Annual Demand Projections <a class="anchor" id="annual_demands"></a>
The <a href="https://www.nrel.gov/docs/fy18osti/71500.pdf" rel="nofollow">Electrification Futures Study (EFS) Scenarios of Electric Technology Adoption and Power Consumption for the United States</a> report was published in 2018 and presents a range of demand scenarios for all the energy sectors in the United States. Specifically, the study estimates building-related service demands, as shown in Table 1, primarily by downscaling service demands data from the US EIA 2017 Annual Energy Outlook and the NEMS model.

EIA service demand data were made available by US Census Division to the EFS study. These Census Division-level data were then downscaled to the state-level in the EFS by using factors relevant to each end-use demand. For example, residential space heating service demands at the Census Division were downscaled to states by taking the product of each state's share of heating-degree-days and residential square footage within a given region. Similarly, residential water heating was downscaled by using the number of households. These state-level demand projectons to 2050 were made available by the [EFS](https://data.nrel.gov/submissions/92), for the demand types listed in Table 1.

These state-level demands are aggregated, by demand category, to the none OEO regions shown in Figure 1 of the Overview notebook. Using these aggregated data, the fraction of demand by category in each OEO region are estimated for every year. For example, residential space heating demand in the Northeast region is almost 20% of US total demand in 2050.

The national level end-use demands, drawn from the US EPA MARKAL database [(Lenox, et al., 2013)](https://cfpub.epa.gov/si/si_public_record_report.cfm?Lab=NRMRL&dirEntryId=278925), were then scaled to each of the nine OEO regions using the regional demand fractions estimated using the EFS. The demand categories in the EFS were mapped to the OEO categories, as shown in Table 1.

**Table 1.** Annual service demands categories in the EFS, mapped to the OEO demands

| EFS Demand Category | Unit | OEO Demand Category |
|-|-|-|
| Commercial air conditioning<img width=150/> | TERABTU |commercial space cooling demand<img width=500/>|
| Commercial cooking | TERABTU | commercial   cooking demand |
| Commercial lighting | GIGALUMEN_YEAR | commercial lighting demand, commercial other electric demand, commercial electric demand for office equipment, commercial other residual fuel oil demand, Other demand in commercial sector |
| Commercial refrigeration | TERABTU | commercial   refrigeration demand |
| Commercial space heating | TERABTU | commercial space   heating demand |
| Commercial ventilation | GIGACUBIC_FOOT | commercial   ventilation demand |
| Commercial water heating | TERABTU | commercial water   heating demand |
| Residential air conditioning | MMBTU | residential   space cooling demand |
| Residential clothes drying | LB | Other demand in   resindetial sector |
| Residential clothes washing | CUBIC_FOOT_CYCLE |  |
| Residential cooking | MMBTU |  |
| Residential dishwashing | CYCLE | residential   other electric demand |
| Residential freezing | CUBIC_FEET | residential   freezing demand |
| Residential lighting | KILOLUMEN_HOUR | residential   lighting demand |
| Residential refrigeration | CUBIC_FEET | residential   refrigeration demand |
| Residential space heating | MMBTU | residential   space heating demand |
| Residential water heating | MMBTU | residential   water heating demand |


End-use regional OEO demand projections can be explored in the tool below.

In [ ]:
def show_demand(con):

    display_types = ['table', 'figure']

    df_demand = pd.read_sql("SELECT * FROM Demand WHERE demand_comm in (SELECT comm_name FROM commodities WHERE flag='d') \
                            AND demand_comm in (SELECT output_comm FROM Efficiency WHERE tech IN (SELECT tech FROM technologies WHERE sector='transport'))", con)
   
    df_tech_desc = pd.read_sql("SELECT comm_name, comm_desc FROM commodities", con)
    df_demand['demand_comm'] = df_demand['demand_comm'].map(lambda x: df_tech_desc.loc[df_tech_desc.comm_name==x,'comm_desc'].values[0].replace('#','').strip())

    demand_comms = []
    for unique_demand_comm in df_demand.demand_comm.unique():
        demand_comms.append(unique_demand_comm)
    demand_comms = list(set(demand_comms))
    regions = df_demand.regions.unique()


    def filter_demand_comm(demand_comm='', o_format=''):
        #demand_comm_name = pd.read_sql("SELECT comm_desc FROM commodities where comm_name=='" + demand_comm +"'", con)
        df_sel = df_demand[(df_demand.demand_comm==demand_comm)]
        df_sel = df_sel.pivot_table(
            index=['regions', 'demand_comm', 'demand_units'],
            columns='periods',
            values='demand').reset_index().set_index('regions')
        #df_sel['demand_comm'] = demand_comm_name.iloc[0,0].replace('#','').strip()
        df_sel.rename(columns={'demand_units': 'units', 'demand_comm':'demand'}, inplace=True)
        df_sel.units = df_sel.units.str.replace('#','').str.strip()
        if o_format == 'table':
            display(
                HTML(
                    tabulate.tabulate(df_sel,
                                      ['region'] + list(df_sel.columns.values),
                                      floatfmt=".1f",
                                      tablefmt='html')))
        elif o_format == 'figure':
            fig, ax = plt.subplots(figsize=(10, 6))
            df_sel.drop(columns=['demand', 'units']).transpose().plot.bar(stacked=True, ax = ax)
            plt.legend(bbox_to_anchor=(0, 1), loc='upper left', ncol=1)
            handles, labels = ax.get_legend_handles_labels()
            ax.legend(handles[::-1], labels[::-1], loc='upper right',bbox_to_anchor=(1.2, 1))
            plt.ylabel(demand_comm_name.iloc[0,0].replace('#','').strip() + ' (' + df_sel.units.unique()[0].replace('#','') + ')')
            plt.xlabel('')
    w2 = widgets.Select(options=demand_comms, description='demand')
    w3 = widgets.Select(options=display_types)

    w_filter = widgets.interactive(filter_demand_comm, demand_comm=w2, o_format=w3)
    controls_rows(w_filter)

display(Markdown('Annual end-use regional demand projections to 2050'))
show_demand(con)


#####  1.1.2 Hourly Demand Representation <a class="anchor" id="hourly_demands"></a>
These service demands are not specified at the temporal resolution currently proposed for the OEO, where each model year is represented by a few days, at an hourly level.

Therefore, we use regional electricity demand projections from the PowerGenome model (more details in the Electric Sector notebook) to apportion the annual demands to the hourly time scale across the representative days within the model. This apportionment is done for the time-varying demand categories in the residential and commercial sectors shown in Table 2.

In [ ]:
display(Markdown('**Table 2.** Demand categories that have hourly resolution in the OEO'))
df = pd.read_sql("SELECT comm_name, comm_desc FROM commodities \
	WHERE comm_name IN \
	(SELECT DISTINCT(demand_name) FROM DemandSpecificDistribution)", con)
df.columns=['Demand','Description']
df['Description'] = df['Description'].str.replace('#','').str.strip()
display(HTML(tabulate.tabulate(df.set_index('Demand'),df.columns,tablefmt='html')))

**We are planning to update this approach in the short-term.** NREL has developed and published [hourly level electricity demand profiles](https://data.nrel.gov/submissions/126) (for 8760 hours a year) for a few of the demand categories listed in Table 1 (space heating and cooling, water heating, clothes and dish washing/drying). NREL has used the outputs of models such as ResStock and ComStock and data (e.g. from metering studies, EPRI) to develop these representative hourly profiles for a range of future scenarios. Although these have been developed for electricity demands only, we assume for now that service demands follow the same profiles. We will select the electrification profiles from the 'High Electrification, Rapid End-use Technology Advancement' scenario for our estimates, since it is likely most representative of total service demands. For the subsectors for which no hourly profiles are available, we will assume service demands are constant throughout the year.

In [ ]:
display(Markdown('The downscaled hourly demand profiles for residential/commercial end use demand are shown below. Refer to Table 2 for a description of the technologies'))

def show_hourly_dem(con):
    df = pd.read_sql("SELECT * FROM DemandSpecificDistribution", con)
    regions = df.regions.unique()

    def select_region(region=''):
        df_sel = df[df.regions==region].copy()
        fig, ax = plt.subplots(figsize=(10,6))
        for dem in df_sel.demand_name.unique():
            df_sel_dem = df_sel[df_sel.demand_name==dem]
            plt.plot(np.arange(0, len(df_sel_dem['dds'])),df_sel_dem['dds'], label=dem)
        plt.ylabel('Hourly demand fraction')
        plt.xlabel('Hour')
        plt.legend(loc='upper right',bbox_to_anchor=(1.2, 1))

    w1 = widgets.Select(options=regions)
    w = widgets.interactive(select_region, region=w1)

    controls_rows(w)
show_hourly_dem(con)

#### 1.2 Technology/commodity description lookup tool <a class="anchor" id="description_look_up"></a>
Use the tool below to search for any key words that may describe a technology or commodity of interest (e.g. heating, cooling). The tool provides a list of all the technologies in the database that may be relevant to the query.

In [2]:
w = widgets.Text(value='space heating demand')
display(w)
def f(w):
    if len(w)>0:
        df1 = pd.read_sql("SELECT * FROM commodities WHERE comm_desc LIKE '%" + w + "%'", con)
        df1['desc'] = df1['comm_desc'].str.replace('#','').str.strip()
        df1['comm_tech'] = df1['comm_name']
        df1['type'] = 'commodity'

        df2 = pd.read_sql("SELECT * FROM technologies WHERE tech_desc LIKE '%" + w + "%'", con)
        df2['desc'] = df2['tech_desc'].str.replace('#','').str.strip()
        df2['comm_tech'] = df2['tech']
        df2['type'] = 'technology'


        df = pd.concat([df1[['comm_tech','type','desc']], df2[['comm_tech','type','desc']]])
        
        if len(df)>0:
            display(HTML(tabulate.tabulate(df.set_index('comm_tech'),['technology/commodity','type','description'],tablefmt='html')))
        else:
            print('')
    else:
        print('')
            


out = widgets.interactive_output(f, {'w': w})
display(out)

Text(value='space heating demand')

Output()

#### 1.3 Network diagram lookup tool  <a class="anchor" id="network_look_up"></a>
Use the [description lookup tool](#description_look_up) above to identify specific commodity or technology names. Type the name in the box below to generate a corresponding network diagram for that commodity or technology. The slider can be used to view different upstream levels of the network diagram.

In [3]:
w = widgets.Text(value='RSH')
display(w)
def f(w):
    if len(w)>0:
        
        df1 = pd.read_sql("SELECT comm_name, comm_desc FROM commodities WHERE comm_name='" + w + "'", con)
        df1['desc'] = df1['comm_desc'].str.replace('#','').str.strip()
        df1['comm_tech'] = df1['comm_name']
        df1['type'] = 'commodity'

        df2 = pd.read_sql("SELECT * FROM technologies WHERE tech='" + w + "'", con)
        df2['desc'] = df2['tech_desc'].str.replace('#','').str.strip()
        df2['comm_tech'] = df2['tech']
        df2['type'] = 'technology'

        df = pd.concat([df1[['comm_tech','type','desc']], df2[['comm_tech','type','desc']]])

    
        if len(df)>0:
            def show_desc(level):
                display(Markdown(df['desc'][0]))
                final_dem = df['comm_tech'][0]
                df_graph = return_flowd_table(final_dem,level)
                args = create_args_flowd(df_graph)
                colors, quick_run_dot_fmt = return_format_colors()
                args.update(colors)
                #o_str = 'rankdir = "LR" ;'
                #r_str = 'rankdir = "LR" ; \n\t size="12,12";'
                #quick_run_dot_fmt = quick_run_dot_fmt.replace(o_str, r_str)
                dot_graph = quick_run_dot_fmt % args
                display(graphviz.Source(dot_graph))
            w2 = widgets.IntSlider(value=1,min=0,max=10,step=1,description='Level:',disabled=False,continuous_update=True,orientation='horizontal',readout=True,readout_format='d')
            w = widgets.interactive(show_desc, level=w2)
            controls_rows(w)
        else:
            print('')

    else:
        print('')
            


out = widgets.interactive_output(f, {'w': w})
display(out)

Text(value='RSH')

Output()

#### 1.4 Technology/commodity look-up tool <a class="anchor" id="lookup"></a>
Use the tool below to retrieve the description for any technology or commodity within the database. Type the commodity or technology name in the box below to view the description. Note that names are case sensitive.

In [4]:
w = widgets.Text(value='RSH')
display(w)
def f(w):
    df = pd.read_sql("SELECT * FROM commodities WHERE comm_name='" + w + "'", con)
    if len(df)==0:
        df = pd.read_sql("SELECT * FROM technologies WHERE tech='" + w + "'", con)

    if len(df)>0:
        try:
            display(Markdown((df['comm_desc'].values[0].replace('#', '').strip())))
        except:
            display(Markdown(df['tech_desc'].values[0].replace('#', '').strip()))
    else:
        print('')


out = widgets.interactive_output(f, {'w': w})
display(out)


Text(value='RSH')

Output()

#### 1.5 Demand Technology Specification <a class="anchor" id="demand_technology"></a>

The specification of demand technologies within the residential and commercial sectors is based on the US EPA MARKAL database [(Lenox, et al., 2013)](https://cfpub.epa.gov/si/si_public_record_report.cfm?Lab=NRMRL&dirEntryId=278925). Techno-economic parameters for an individual technology are assumed to be the same for all regions. These technologies only have investment costs associated with them (no O\&M). Currently, existing capacity has been divided equally among the nine regions, however, this will be revisited in the near-term, using the state-level representation in the MARKAL database, to aggregate to the OEO regions.

Input values drawn directly from the database can be viewed below. Regarding discount rates, all existing capacity is given a technology-specific discount rate of 30%. Without this specification, Temoa's least cost optimization results in the early replacement of virtually all existing demand technologies with new capacity at higher efficiency. Thus this 30% rate is used to ensure that existing capacity is utilized for the remainder of its lifetime under base case assumptions.


##### 1.5.1 Investment costs <a class="anchor" id="costinvest"></a>

In [5]:
def show_cost_invest(con):
    
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)

    df = pd.read_sql(
        "SELECT regions, tech,vintage, cost_invest, cost_invest_units FROM CostInvest WHERE tech IN (SELECT tech FROM technologies WHERE sector=='residential' OR sector='commercial' ) ORDER BY tech, vintage",
        con)

    display_types = ['table', 'figure']
    
    df['agg_tech'] = df['tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())
    techs = []
    for unique_tech in df.agg_tech.unique():
        try:
            int(unique_tech.split('_')[-1])
            techs.append('_'.join(unique_tech.split('_')[0:-1]))
        except:
            techs.append(unique_tech)
    techs = ['All'] + list(set(techs))
    regions = df.regions.unique()


    def filter_tech(tech='', region='', o_format=''):
        if tech == 'All':
            df_sel = df[(df.regions == region)]
        else:
            df_sel = df[(df.agg_tech.str.contains(tech)) & (df.regions == region)]
        if o_format == 'table':
            df_sel = df_sel.pivot_table(
                index=['regions', 'tech', 'agg_tech', 'cost_invest_units'],
                columns='vintage',
                values='cost_invest').reset_index().set_index('regions')
            df_sel.rename(columns={'cost_invest_units': 'units', 'agg_tech':'description'}, inplace=True)
            df_sel['units'] = df_sel['units'].str.replace('#','').str.replace('M$','$M').str.strip()
            if len(df_sel['units'].unique())==1:
                df_sel['units'] = df_sel['units'].unique()[0].replace('M$','$M')
            display(
                HTML(
                    tabulate.tabulate(df_sel,
                                      ['region'] + list(df_sel.columns.values),
                                      floatfmt=".0f",
                                      tablefmt='html')))
        elif (o_format == 'figure') & (tech!='All') :
            fig, ax = plt.subplots(figsize=(10, 6))
            for ind_tech in df_sel.tech.unique():
                plt.plot(df_sel[df_sel.tech == ind_tech].vintage,
                         df_sel[df_sel.tech == ind_tech].cost_invest,
                         label=ind_tech)
            plt.legend()
            plt.ylabel('Investment costs (' + df_sel['cost_invest_units'].unique()[0].replace('#','').strip().replace('M$','$M') + ')')
            #plt.ylim([0, df.cost_invest.max() * 1.1])
            plt.xlabel('Vintage')


    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w3 = widgets.Select(options=display_types)
    w = widgets.interactive(filter_tech, tech=w1, region=w2, o_format=w3)

    controls_rows(w)
    
show_cost_invest(con)

##### 1.5.2 Efficiency <a class="anchor" id="efficiency"></a>

In [6]:
def show_efficiency(con):
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)
    df = pd.read_sql("SELECT regions, tech, vintage, efficiency FROM Efficiency WHERE tech IN (SELECT tech FROM technologies WHERE sector=='residential' OR sector='commercial')", con)
    df.loc[:,'agg_tech'] = df.loc[:,'tech'] #[map_plants[y] for x in df.tech for y in map_plants.keys() if y.lower() in x.lower()] #map agg technologies
    df_sum = df#df.drop("vintage", axis=1).groupby(by = ['regions','agg_tech']).sum().reset_index()
    df_sum['agg_tech'] = df_sum['agg_tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())
    techs = ['All'] + list(df_sum.agg_tech.unique())
    regions = df_sum.regions.unique()
    def filter_tech(tech ='', region = ''):
        if tech=='All':
            df_sel = df_sum[(df_sum.regions==region)]
        else:
            df_sel = df_sum[(df_sum.agg_tech==tech) & (df_sum.regions==region)]
            df_sel.efficiency = 100*df_sel.efficiency
        df_sel = df_sel[['regions','tech','agg_tech','vintage','efficiency']]
        display(HTML(tabulate.tabulate(df_sel.set_index('regions'), ["regions", "technology", "description","vintage",'efficiency (%)' ], floatfmt=".1f", tablefmt='html')))

    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w = widgets.interactive(filter_tech, tech=w1, region=w2)

    controls_rows(w)
    
show_efficiency(con)

##### 1.5.3 Existing Capacity <a class="anchor" id="exist_cap"></a>

In [7]:
def show_exist_cap(con):
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)
    df = pd.read_sql("SELECT regions, tech,vintage, exist_cap, exist_cap_units FROM ExistingCapacity WHERE tech IN (SELECT tech FROM technologies WHERE sector=='residential' OR sector='commercial')", con)
    df.loc[:,'agg_tech'] = df.loc[:,'tech'] #[map_plants[y] for x in df.tech for y in map_plants.keys() if y.lower() in x.lower()] #map agg technologies
    df_sum = df.drop("vintage", axis=1).groupby(by = ['regions','tech','agg_tech','exist_cap_units']).sum().reset_index()
    df_sum.sort_values(by='exist_cap', ascending=False, inplace=True)
    df_sum['agg_tech'] = df_sum['agg_tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())
    techs = ['All'] + list(df_sum.agg_tech.unique())
    regions = df_sum.regions.unique()
    def filter_tech(tech ='', region = ''):
        if tech=='All':
            df_sel = df_sum[(df_sum.regions==region)]
        else:
            df_sel = df_sum[(df_sum.agg_tech==tech) & (df_sum.regions==region)]
        display(HTML(tabulate.tabulate(df_sel.set_index('regions'), ["regions", "technology", "description", "units","capacity" ], floatfmt=".1f", tablefmt='html')))
    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w = widgets.interactive(filter_tech, tech=w1, region=w2)

    controls_rows(w)
    
show_exist_cap(con)


##### 1.5.4 Discount Rate <a class="anchor" id="disc_rates"></a>

In [8]:
def show_disc_rate(con):
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)
    df = pd.read_sql("SELECT regions, tech, vintage, tech_rate FROM DiscountRate \
    WHERE tech in (SELECT tech FROM technologies WHERE sector='residential' OR sector='commercial')", con)
    df['agg_tech'] = df['tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())
    techs = []
    for unique_tech in df.agg_tech.unique():
        techs.append(unique_tech)
    techs = ['All'] + list(set(techs))
    regions = df.regions.unique()
    def filter_tech(tech='', region =''):
        if tech=='All':
            df_sel = df[(df.regions==region)]
        else:
            df_sel = df[(df.agg_tech==tech) & (df.regions==region)]
        df_sel = df_sel.pivot_table(index=['regions','tech', 'agg_tech'], columns='vintage', values='tech_rate').reset_index()
        df_sel.rename(columns={'agg_tech':'description'},inplace=True)
        header = list(df_sel.columns.values)
        display(HTML(tabulate.tabulate(df_sel.set_index('regions'), header, floatfmt=".3f", tablefmt='html')))
    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w = widgets.interactive(filter_tech, tech=w1, region=w2)

    controls_rows(w)
    
show_disc_rate(con)